##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Recuperación de preguntas y respuestas del codificador de oraciones universal multilingüe


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td data-parent-segment-id="12900598">     <a href="https://tfhub.dev/s?q=google%2Funiversal-sentence-encoder-multilingual-qa%2F3%20OR%20google%2Funiversal-sentence-encoder-qa%2F3"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos de TF Hub</a>
</td>
</table>

Esta es una demostración del uso del [modelo de preguntas y respuestas multilingüe del codificador universal](https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3) para la recuperación de preguntas y respuestas de texto, que ilustra el uso de **question_encoder** y **response_encoder** del modelo. Usamos oraciones de párrafos de [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) como conjunto de datos de la demostración, cada oración y su contexto (el texto que acompaña la oración) se codifica en incorporaciones de alta dimensión con el **response_encoder**. Estas incorporaciones se almacenan en un índice generado con la biblioteca [simpleneighbors](https://pypi.org/project/simpleneighbors/) para la recuperación de preguntas y respuestas.

Durante la recuperación, se selecciona una pregunta aleatoria del conjunto de datos [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) y se codifica en una incorporación de alta dimensión con **question_encoder** y se consulta el índice de simpleneighbors que devuelve una lista aproximada de vecinos más cercanos en el espacio semántico.

### Más modelos

Puede encontrar todos los modelos de incorporación de texto alojados actualmente [aquí](https://tfhub.dev/s?module-type=text-embedding) y también todos los modelos que se entrenaron con SQuAD [aquí](https://tfhub.dev/s?dataset=squad).

## Preparación


In [ ]:
%%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install -q "tensorflow-text==2.11.*"
!pip install -q simpleneighbors[annoy]
!pip install -q nltk
!pip install -q tqdm

In [ ]:
#@title Setup common imports and functions
import json
import nltk
import os
import pprint
import random
import simpleneighbors
import urllib
from IPython.display import HTML, display
from tqdm.notebook import tqdm

import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

nltk.download('punkt')


def download_squad(url):
  return json.load(urllib.request.urlopen(url))

def extract_sentences_from_squad_json(squad):
  all_sentences = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      sentences = nltk.tokenize.sent_tokenize(paragraph['context'])
      all_sentences.extend(zip(sentences, [paragraph['context']] * len(sentences)))
  return list(set(all_sentences)) # remove duplicates

def extract_questions_from_squad_json(squad):
  questions = []
  for data in squad['data']:
    for paragraph in data['paragraphs']:
      for qas in paragraph['qas']:
        if qas['answers']:
          questions.append((qas['question'], qas['answers'][0]['text']))
  return list(set(questions))

def output_with_highlight(text, highlight):
  output = " "
  i = text.find(highlight)
  while True:
    if i == -1:
      output += text
      break
    output += text[0:i]
    output += ''+text[i:i+len(highlight)]+''
    text = text[i+len(highlight):]
    i = text.find(highlight)
  return output + "\n"

def display_nearest_neighbors(query_text, answer_text=None):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)

  if answer_text:
    result_md = '''
    Random Question from SQuAD:
      %s
    Answer:
      %s
    ''' % (query_text , answer_text)
  else:
    result_md = '''
    Question:
      %s
    ''' % query_text

  result_md += '''
    Retrieved sentences :
    
  '''

  if answer_text:
    for s in search_results:
      result_md += output_with_highlight(s, answer_text)
  else:
    for s in search_results:
      result_md += '' + s + '\n'

  result_md += ""
  display(HTML(result_md))

Ejecute el siguiente bloque de código para descargar y extraer el conjunto de datos SQuAD en:

- **oraciones** es una lista de tuplas (texto, contexto): cada párrafo del conjunto de datos SQuAD se divide en oraciones mediante el uso de la biblioteca nltk. El texto de la oración y del párrafo forman la tupla (texto, contexto).
- **preguntas** es una lista de tuplas (pregunta, respuesta).

Nota: Puede usar esta demostración para indexar el conjunto de datos de entrenamiento de SQuAD o el conjunto de datos de desarrollador más pequeño (1.1 o 2.0) al seleccionar **squad_url** a continuación.


In [ ]:
#@title Download and extract SQuAD data
squad_url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json' #@param ["https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"]

squad_json = download_squad(squad_url)
sentences = extract_sentences_from_squad_json(squad_json)
questions = extract_questions_from_squad_json(squad_json)
print("%s sentences, %s questions extracted from SQuAD %s" % (len(sentences), len(questions), squad_url))

print("\nExample sentence and context:\n")
sentence = random.choice(sentences)
print("sentence:\n")
pprint.pprint(sentence[0])
print("\ncontext:\n")
pprint.pprint(sentence[1])
print()

En el siguiente bloque de código, se configura el gráfico de tensorflow **g** y **session** con las signaturas <strong>question_encoder</strong> y **response_encoder** del <a>modelo de preguntas y respuestas multilingüe del codificador universal</a>.

In [ ]:
#@title Load model from tensorflow hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3", "https://tfhub.dev/google/universal-sentence-encoder-qa/3"]
model = hub.load(module_url)


En el siguiente bloque de código, se calculan las incorporaciones de todas las tuplas de texto y contexto y se almacenan en un índice de [simpleneighbors](https://pypi.org/project/simpleneighbors/) con el **response_encoder**.


In [ ]:
#@title Compute embeddings and build simpleneighbors index
batch_size = 100

encodings = model.signatures['response_encoder'](
  input=tf.constant([sentences[0][0]]),
  context=tf.constant([sentences[0][1]]))
index = simpleneighbors.SimpleNeighbors(
    len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(sentences))
slices = zip(*(iter(sentences),) * batch_size)
num_batches = int(len(sentences) / batch_size)
for s in tqdm(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
    input=tf.constant(response_batch),
    context=tf.constant(context_batch)
  )
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(sentences))


Durante la recuperación, se codifica la pregunta con **question_encoder** y se usa la incorporación de preguntas para consultar el índice de simpleneighbors.

In [ ]:
#@title Retrieve nearest neighbors for a random question from SQuAD
num_results = 25 #@param {type:"slider", min:5, max:40, step:1}

query = random.choice(questions)
display_nearest_neighbors(query[0], query[1])